# Bloqade-Python tutorial 

Author: Hyeok Kim

"Bloqade is an SDK designed to make writing and analyzing the results of analog quantum programs on QuEra's neutral atom quantum computers as seamless and flexible as possible."

Official documentation: https://bloqade.quera.com/latest/

This tutorial aims to introduce how to arrange atoms, ...

In [33]:
import numpy as np
from bloqade import var, start, get_capabilities
from bloqade.atom_arrangement import Chain, Square, Honeycomb, Kagome, Lieb, Rectangular, Triangular
from bloqade import visualization as bv
import time
import pprint

## Arrange atoms!

Bloqade affers different custom and pre-defined configuration methods for atom positions.

In [2]:
# Custom arragement methods
custom_atoms = start.add_position((0,0)).add_position([(3.1, 0.0), (4.1, 2.2)])
custom_atoms

                                   Atom Positions                               
    ┌──────────────────────────────────────────────────────────────────────────┐
2.20┤                                                                         •│
    │                                                                          │
    │                                                                          │
1.83┤                                                                          │
    │                                                                          │
    │                                                                          │
1.47┤                                                                          │
    │                                                                          │
    │                                                                          │
1.10┤                                                                          │
    │                       

In [3]:
# Predefined arragement methods
# Chain(3) # > n
# Square(3) # > (n^2)
# Honeycomb(3) # > 2*(n^2)
# Kagome(3) # > 3*(n^2)
# Lieb(2) # > 3 * (n^2)
# Rectangular(3, 2) # > n * m
# Triangular(3) # > (n^2)

atoms = Square(3)
print(atoms)

                                   Atom Positions                               
    ┌──────────────────────────────────────────────────────────────────────────┐
2.00┤•                                    •                                   •│
    │                                                                          │
    │                                                                          │
1.67┤                                                                          │
    │                                                                          │
    │                                                                          │
1.33┤                                                                          │
    │                                                                          │
    │                                                                          │
1.00┤•                                    •                                   •│
    │                       

### Simulate defected atoms

In [4]:
atoms.apply_defect_count(2)

                                   Atom Positions                               
    ┌──────────────────────────────────────────────────────────────────────────┐
2.00┤ •• vacant                           •                                   •│
    │                                                                          │
    │                                                                          │
1.67┤                                                                          │
    │                                                                          │
    │                                                                          │
1.33┤                                                                          │
    │                                                                          │
    │                                                                          │
1.00┤•                                    •                                   •│
    │                       

In [5]:
atoms.apply_defect_density(0.2)

                                   Atom Positions                               
    ┌──────────────────────────────────────────────────────────────────────────┐
2.00┤ •• vacant                           •                                   •│
    │                                                                          │
    │                                                                          │
1.67┤                                                                          │
    │                                                                          │
    │                                                                          │
1.33┤                                                                          │
    │                                                                          │
    │                                                                          │
1.00┤•                                    •                                   •│
    │                       

In [6]:
atoms.apply_defect_density(0.2).show()

### To check informations about the atoms

In [7]:
print("# atoms  \t", atoms.n_atoms)
print("# dims   \t", atoms.n_dims)
print("# sites \t", atoms.n_sites)
print("# vacant sites \t", atoms.n_vacant)
# n_atoms + n_vacant = n_sites

# atoms  	 9
# dims   	 2
# sites 	 9
# vacant sites 	 0


## Manipulate atoms–Write a program!

### Syntax

#### Overall structure
`Circuit.Drive.Field1(.Field2).Qubits.WaveformFunctions`

#### Breakdown
- Drive
  - `hyperfine`: a qubit in a hyperfine state has a longer term stability, commonly used in ion-trap QCs.
  - `rydberg`: (oversimplified) an atom excited to a high-energy state such that prevents other nearby atoms from being excited (i.e., blockade effect), enabling the entanglement between two Rydberg qubits in a short distance (i.e., energy penalty; two atoms further than 8 𝜇m has low interaction; but ideally like 3-4). 
- Field: "A "field" is a summation of one or more "drives", with a drive being the sum of a waveform and spatial modulation."
  - Rabi field: a qubit oscillates between |0> (ground) and |1> (rydberg/hyperfine)
    - `rabi.amplitude` (real-valued): affecting Rabi frequency at which a qubit oscillates between the two states.
    - `Rabi.phase` (real-valued): "the time offset of the laser drive ... This sets the direction on the Bloch sphere around which the qubit is driven."
  - `detuning` field: "how off-resonant the laser is from the atomic energy transitions" > then it sets the range of the oscillation of the qubit (resonance: the drive's frequency == atom's natural frequency). When detuning is large-negative, the atom prefers the ground state; and when it is large-positive, the atom prefers the Rydberg state.
- Qubit selection
  - `uniform`: affecting all atoms
  - `location(locations, scales)`: affecting atoms in the specified `locations` (in index) with individual `scales`.
  - `scale(coeffs)`: all atoms with individual scale factors (note: order-sensitive)
- Waveform functions
  - `linear(start, stop, duration)` : to apply a linear waveform
  - `constant(value, duration)` : to apply a constant waveform
  - `poly([coefficients], duration)` : to apply a polynomial waveform
  - `apply(wf:bloqade.ir.Waveform)`: to apply a pre-defined waveform
  - `piecewise_linear([durations], [values])`: to apply a piecewise linear waveform
  - `piecewise_constant([durations], [values])`: to apply a piecewise constant waveform
  - `fn(f(t,..))`: to apply a function as a waveform
  - To help understanding:
    * Linear: a linear transformation
    * Constant: a scalar multiplied
    * Piece-wise linear: the value `duration[i]` is of the linear segment between `values[i]` and `values[i+1]`.

Note: actual value ranges for fields (amplitude, phase, and detuning) are contingent on the device in use.

Take a look: https://arxiv.org/pdf/2306.11727
Also: https://www.quera.com/events/hamiltonian-simulation-on-queras-256-qubit-aquila-machine 

In [8]:
import altair as alt
import pandas as pd

In [9]:
def durToPoints(dur):
    return [0] + np.cumsum(np.array(dur)).tolist()

In [10]:
dur_val_data = pd.DataFrame({
  'adiabatic_durations': durToPoints([0.4, 3.2, 0.4]),
  'amplitude_values': [0, 15.8, 15.8, 0]
})

alt.Chart(title="How piecewise linear function works with durations", data=dur_val_data).mark_line().encode(
    x='adiabatic_durations',
    y='amplitude_values'
)

alt.Chart(...)

In [11]:
# the example is from the official doc
adiabatic_durations = [0.4, 3.2, 0.4]

max_detuning = var("max_detuning")
max_rabi = var("max_rabi")

adiabatic_program = (
    # `lattice_spacing` is for the distance between atom centers. 
    # "lattice_spacing" is a variable, it has no value yet, but is assigned later
    Square(3, lattice_spacing="lattice_spacing") 
    .rydberg.rabi.amplitude.uniform.piecewise_linear(
        durations=adiabatic_durations, values=[0.0, max_rabi, max_rabi, 0.0]
    )
    .detuning.uniform.piecewise_linear(
        durations=adiabatic_durations,
        values=[
            -max_detuning, # scalar variables support direct arithmetic operations
            -max_detuning,
            max_detuning,
            max_detuning,
        ],
    )
    # value assignment: a single value -> a single variable 
    .assign(max_rabi=15.8, max_detuning=16.33) 
    # value assignment: multiple values -> a single variable (so that the parameter changes over time i.e., parameter sweep)
    .batch_assign(lattice_spacing=np.arange(4.0, 7.0, 0.5))
)

In [12]:
adiabatic_program.parse_circuit()

AnalogCircuit
├─ register
│  ⇒ AtomArrangement
│    ├─ Location: filled
│    │  ├─ x
│    │  │  ⇒ Literal: 0
│    │  └─ y
│    │     ⇒ Literal: 0
│    ├─ Location: filled
│    │  ├─ x
│    │  │  ⇒ Literal: 0
│    │  └─ y
│    │     ⇒ Variable: lattice_spacing
│    ├─ Location: filled
│    │  ├─ x
│    │  │  ⇒ Literal: 0
│    │  └─ y
│    │     ⇒ *
│    │       ├─ Variable: lattice_spacing
│    │       └─ Literal: 2
│    ├─ Location: filled
│    │  ├─ x
│    │  │  ⇒ Variable: lattice_spacing
│    │  └─ y
│    │     ⇒ Literal: 0
│    ├─ Location: filled
│    │  ├─ x
│    │  │  ⇒ Variable: lattice_spacing
│    │  └─ y
│    │     ⇒ Variable: lattice_spacing
│    ├─ Location: filled
│    │  ├─ x
│    │  │  ⇒ Variable: lattice_spacing
│    │  └─ y
│    │     ⇒ *
│    │       ├─ Variable: lattice_spacing
│    │       └─ Literal: 2
│    ├─ Location: filled
│    │  ├─ x
│    │  │  ⇒ *
│    │  │    ├─ Variable: lattice_spacing
│    │  │    └─ Literal: 2
│    │  └─ y
│    │     ⇒ Literal: 0
│    

In [13]:
# Bug: when try to show a program, the "assign" method doesn't work!
(Square(3, lattice_spacing="lattice_spacing") 
.rydberg.rabi.amplitude.uniform.piecewise_linear(
    durations=adiabatic_durations, values=[0.0, 15.8, 15.8, 0.0]
)
.detuning.uniform.piecewise_linear(
    durations=adiabatic_durations,
    values=[
        -16.33, # scalar variables support direct arithmetic operations
        -16.33,
        16.33,
        16.33,
    ],
)
.batch_assign(lattice_spacing=np.arange(4.0, 7.0, 0.5))).show()

#### Assignment (not that kind of)

- `assign`: "Assign values to variables declared previously in the program."
- `batch_assign`: "Assign multiple values to a single variable to create a parameter sweep."

## Run a program!

### Check capabilities

Check: https://bloqade.quera.com/latest/reference/hardware-capabilities/?h=your_capabilities_object

In [59]:
capabilities = get_capabilities()
capabilities.capabilities
pprint.pprint(dict(capabilities.capabilities))

{'lattice': LatticeCapabilities(number_qubits_max=256, area=LatticeAreaCapabilities(width=Decimal('75'), height=Decimal('76')), geometry=LatticeGeometryCapabilities(spacing_radial_min=Decimal('4'), spacing_vertical_min=Decimal('4'), position_resolution=Decimal('0.1'), number_sites_max=256)),
 'rydberg': RydbergCapabilities(c6_coefficient=Decimal('5.42E+6'), global_=RydbergGlobalCapabilities(rabi_frequency_min=Decimal('0E-7'), rabi_frequency_max=Decimal('15.8000000'), rabi_frequency_resolution=Decimal('0.0004000'), rabi_frequency_slew_rate_max=Decimal('250.0000000000000'), detuning_min=Decimal('-125.0000000'), detuning_max=Decimal('125.0000000'), detuning_resolution=Decimal('2E-7'), detuning_slew_rate_max=Decimal('2500.0000000000000'), phase_min=Decimal('-99.0'), phase_max=Decimal('99.0'), phase_resolution=Decimal('5E-7'), time_min=Decimal('0E+5'), time_max=Decimal('4'), time_resolution=Decimal('0.001'), time_delta_min=Decimal('0.05')), local=RydbergLocalCapabilities(detuning_min=Decima

In [14]:
# local emulator
start_time = time.time()
emu_results = adiabatic_program.braket.local_emulator().run(100)
print("--- %s seconds ---" % (time.time() - start_time))

--- 166.71098709106445 seconds ---


### CPU usage
<img src="./cpu-use-bloqade-emulator.png" style="border: 1px solid black; border-radius: 5px;">

### Emulation results

In [15]:
emu_report = emu_results.report()

#### Statistics

In [23]:
# bitstrings observed
emu_report.bitstrings()

[array([[1, 1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 0, 1, 1],
        [1, 1, 1, 1, 1, 1, 0, 1, 0],
        [1, 1, 1, 1, 1, 0, 0, 1, 1],
        [1, 1, 1, 0, 1, 1, 1, 1, 0],
        [1, 1, 1, 0, 1, 1, 1, 1, 0],
        [1, 1, 1, 0, 1, 1, 1, 1, 0],
        [1, 1, 1, 0, 1, 1, 1, 1, 0],
        [1, 1, 1, 0, 1, 1, 1, 1, 0],
        [1, 1, 1, 0, 1, 1, 1, 1, 0],
        [1, 1, 0, 1, 1, 1, 1, 1, 1],
        [1, 1, 0, 1, 1, 1, 1, 0, 1],
        [1, 1, 0, 1, 1, 1, 1, 0, 1],
        [1, 1, 0, 1, 1, 1, 1, 0, 1],
        [1, 1, 0, 1, 1, 1, 1, 0, 1],
        [1, 1, 0, 1, 1, 1, 1, 0, 1],
        [1, 1, 0, 1, 1, 1, 1, 0, 1],
        [1, 1, 0, 1, 1, 1, 0, 1, 1],
        [1, 1, 0, 1, 1, 1, 0, 1, 1],
        [1, 1, 0, 1, 1, 1, 0, 1, 1],
        [1, 1, 0, 1, 1, 1, 0, 1, 1],
        [1, 1, 0, 1, 1, 1, 0, 1, 1],
        [1, 1, 0, 1, 1, 1, 0, 1, 1],
        [1, 1, 0, 1, 1, 1, 0, 1, 1],
        [1, 1, 0, 1, 1, 1, 0, 1, 1],
 

In [21]:
# "the number of times that bitstring was observed in the shots"
emu_report.counts()

[OrderedDict([('110111011', 26),
              ('011111110', 23),
              ('110111101', 6),
              ('111011110', 6),
              ('011110111', 5),
              ('011111101', 5),
              ('110011111', 5),
              ('101111110', 4),
              ('011111011', 3),
              ('101111011', 3),
              ('111111110', 3),
              ('011111111', 2),
              ('010111101', 1),
              ('010111110', 1),
              ('010111111', 1),
              ('101111010', 1),
              ('110011110', 1),
              ('110111111', 1),
              ('111110011', 1),
              ('111111010', 1),
              ('111111011', 1)]),
 OrderedDict([('110111010', 12),
              ('011111010', 10),
              ('110111011', 10),
              ('011111110', 9),
              ('010111011', 8),
              ('010111110', 7),
              ('011111011', 6),
              ('010111010', 5),
              ('010111111', 5),
              ('111111010', 4),
 

In [22]:
emu_report.rydberg_densities()

,0,1,2,3,4,5,6,7,8
task_number,,,,,,,,,
0,0.41,0.08,0.42,0.12,0.00,0.06,0.36,0.12,0.40
1,0.57,0.05,0.55,0.05,0.00,0.04,0.63,0.04,0.56
2,0.77,0.02,0.82,0.02,0.00,0.01,0.82,0.01,0.84
3,0.88,0.03,0.91,0.03,0.00,0.02,0.90,0.03,0.90
4,0.92,0.02,0.91,0.02,0.02,0.04,0.93,0.01,0.93
5,0.98,0.01,0.97,0.01,0.05,0.02,0.97,0.02,0.95


#### Visualization

In [ ]:
emu_report.show()

## Other methods

In [61]:
# Another local emulator --> needs multiprocessor (my computer doesn't support it...)
faster_emu_results = adiabatic_program.bloqade.python().run(10000)

In [63]:
# using Aquila machine by QuEra
hw_results = adiabatic_program.parallelize(24).braket.aquila().run_async(100)